In [ ]:
# We want to use TensorFlow 2.0 in the Eager mode for this demonstration. But this module works as well with the Graph mode.
!pip install -U --pre tensorflow-gpu --quiet

In [ ]:
from __future__ import absolute_import, division, print_function

import functools
import os

from matplotlib import gridspec
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

print("TF Version: ", tf.__version__)
print("TF-Hub version: ", hub.__version__)
print("Eager mode enabled: ", tf.executing_eagerly())
print("GPU available: ", tf.test.is_gpu_available())

TF Version:  1.15.0
TF-Hub version:  0.7.0
Eager mode enabled:  False
GPU available:  True


In [ ]:
import PIL.Image, PIL

In [ ]:
import os, random, shutil

#Prompting user to enter number of files to select randomly along with directory
source= "/content/drive/My Drive/Colab Notebooks/wikiart/test/"
base_dir = "/content/drive/My Drive/Colab Notebooks/wikiart/"
random.seed(42)
no_of_files= 10

file_name_list = ["Abstract_Expressionism", "Art_Nouveau_Modern", "Baroque", "Color_Field_Painting", "Cubism", "Early_Renaissance", "Expressionism", "Fauvism", "High_Renaissance", "Impressionism", "Mannerism_Late_Renaissance", "Minimalism", "Naive_Art_Primitivism", "Northern_Renaissance", "Pop_Art", "Post_Impressionism", "Realism", "Rococo", "Romanticism", "Ukiyo_e"]
num_styles = len(file_name_list)


In [ ]:
num_styles

20

In [ ]:
def retrieve(source, name=""):
  print("%"*25+"{ "+name+" Images }"+"%"*25)

  x_images = []

  #get 10 random images
  for i in range(no_of_files):
      #Variable random_file stores the name of the random file chosen
      random_file=random.choice(os.listdir(source))
      print("%d} %s"%(i+1,random_file))
      x_images.append(random_file)

  return x_images    

In [ ]:
os.chdir(base_dir)


In [ ]:
hub_handle = 'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2'
hub_module = hub.load(hub_handle)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
#@title Define image loading and visualization functions

def crop_center(image):
  """Returns a cropped square image."""
  shape = image.shape
  new_shape = min(shape[1], shape[2])
  offset_y = max(shape[1] - shape[2], 0) // 2
  offset_x = max(shape[2] - shape[1], 0) // 2
  image = tf.image.crop_to_bounding_box(
      image, offset_y, offset_x, new_shape, new_shape)
  return image

def load_image(image_path, image_size=(256, 256), preserve_aspect_ratio=True):
  """Loads and preprocesses images."""

  # Load and convert to float32 numpy array, add batch dimension, and normalize to range [0, 1].
  print(image_path)
  img = plt.imread(image_path).astype(np.float32)[np.newaxis, ...]
  if img.max() > 1.0:
    img = img / 255.
  if len(img.shape) == 3:
    img = tf.stack([img, img, img], axis=-1)
  img = crop_center(img)
  img = tf.image.resize(img, image_size, preserve_aspect_ratio=True)

  
  return img

def show_n(images, titles=('',)):
  n = len(images)
  image_sizes = [image.shape[1] for image in images]
  w = (image_sizes[0] * 6) // 320
  plt.figure(figsize=(w  * n, w))
  gs = gridspec.GridSpec(1, n, width_ratios=image_sizes)
  for i in range(n):
    plt.subplot(gs[i])
    plt.imshow(images[i][0], aspect='equal')
    plt.axis('off')
    plt.title(titles[i] if len(titles) > i else '')
  plt.show()


In [ ]:
def tensor_to_image(tensor):
  print('hitin 1')
  tensor = tensor*255
  print('hitin 2')
  tensor1 = np.array(tensor, dtype=np.uint8)
  print('hitin 3')
  if np.ndim(tensor1)>3:
    assert tensor1.shape[0] == 1
    tensor1 = tensor1[0]
  print('hit complete')
  return PIL.Image.fromarray(tensor1)

In [ ]:
for style in file_name_list:
  #get 10 style images
  try:
    style_images = retrieve((source+style), "style")
  except:
    print("retrive err")

  for content in file_name_list:
    #get 10 content images
    try:
      content_images = retrieve((source+content), "content")
    except:
      print("retrive err")

    if not (style == content):
      new_file = content+"|"+style
      try:
        os.mkdir(new_file)
        os.chdir(new_file)
        for i in range(no_of_files):
          #create the images
          content_image = load_image(source + content +"/"+ content_images[i])
          style_image = load_image(source + style +"/"+ style_images[i])
          style_image = tf.nn.avg_pool(style_image, ksize=[3,3], strides=[1,1], padding='SAME') 
          pastiche = hub_module(tf.constant(content_image), tf.constant(style_image))[0]

          print("Created Image")
          img = tensor_to_image(pastiche)
          print("hit1")
          img.save(new_file+" "+str(i)+".jpg")
          print("hit1")
      except:
        print("Something went wrong when writing to the file")
      
    os.chdir(base_dir) 

%%%%%%%%%%%%%%%%%%%%%%%%%{ style Images }%%%%%%%%%%%%%%%%%%%%%%%%%
1} sam-francis_exposition-sam-francis-19-mars-5-mai-1983-poster.jpg
2} arthur-pinajian_untitled-landscape-bellport-no-0001-1985.jpg
3} lee-krasner_night-creatures-1965.jpg
4} paul-jenkins_not_detected_235710.jpg
5} joan-miro_not_detected_227976.jpg
6} willem-de-kooning_untitled-2.jpg
7} perle-fine_unequivocably-blue-1967.jpg
8} richard-pousette-dart_intensive-crescent-1985.jpg
9} jack-tworkov_crossfield-i-1968.jpg
10} jane-frank_april-screen-1968.jpg
%%%%%%%%%%%%%%%%%%%%%%%%%{ content Images }%%%%%%%%%%%%%%%%%%%%%%%%%
1} mark-tobey_komposition-schwarz-1972.jpg
2} henri-matisse_the-circus-1947.jpg
3} brice-marden_distant-muses-2000.jpg
4} sam-francis_untitled-4.jpg
5} paul-reed_gje-2004.jpg
6} mark-tobey_reclining-figure-of-a-horse-1964.jpg
7} joan-mitchell_diabolo-neige-et-fleurs-1969.jpg
8} brice-marden_etchings-to-rexroth-1-1986.jpg
9} jackson-pollock_number-17-1949.jpg
10} cy-twombly_untitled-bastian-38.jpg
%%%%%%%%%

In [ ]:
#create specific file

style = "Abstract_Expressionism"
content = "Realism"


try:
  style_images = retrieve((source+style), "style")
except:
  print("retrive err")

#show_n(style_images)

try:
  content_images = retrieve((source+content), "content")
except:
  print("retrive err")
#show_n(content_images)

new_file = style+"|"+content


try:
  for i in range(no_of_files):
    #create the images
    try:
      content_image = load_image(source + content +"/"+ content_images[i])
      style_image = load_image(source + style +"/"+ style_images[i])
      style_image = tf.nn.avg_pool(style_image, ksize=[3,3], strides=[1,1], padding='SAME') 
      pastiche = hub_module(tf.constant(content_image), tf.constant(style_image))[0]
      print(type(pastiche))
      print("Created Image")
      print("hit1")
      img = tensor_to_image(pastiche)
      print("hit1")
      img.save(new_file+" "+str(i)+".jpg")
      print("hit1")
    except:
      print("oh no")


except:
  print("Something went wrong when writing to the file")

os.chdir(base_dir) 